<a href="https://colab.research.google.com/github/m-wessler/nbm-verification/blob/main/CoLab_Familiarization_Metadata_Pull.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import
Using the ! as an escape character, we can call pip (python package installer) to download and install necessary modules.<br>
Use the "play" button (or Shift+Enter) to run a cell and proceed one cell at a time. Notebooks are generally meant to be run sequentially, in order, as a python script would be.

In [ ]:
!pip install cartopy

In [ ]:
import json
import requests

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.img_tiles import GoogleTiles

## User Variables
Set user variables by either directly entering values into the python code, or by using the form to the right (recommended). These forms are used to populate variables across multiple scripts in the STID CoLab toolkit.

In [ ]:
token = '' #@param {type:'string'}
cwa = '' #@param {type:'string'}
network = '<select network>' #@param ['<select network>', 'NWS', 'NWS+RAWS', 'NWS+HADS', 'NWS+RAWS+HADS', 'NWS', 'RAWS', 'HADS', 'ALL'] {type:'string'}
variable = '<select variable>' #@param ['<select variable>', 'air_temp' ,'precip', 'humidity', 'wind', 'gust', 'snow', 'cloud'] {type:'string'}

## Script Main
Here we will fetch the data from the Synoptic API, filter it, and move it into a Pandas DataFrame. The following blocks of code will do some basic analysis and produce a set of histograms and a map. Run through each sequentially.

Running blocks of code out of order can cause a number of errors if variables are overwritten. Use "Runtime" > "Restart Session" and "Edit" > "Clear All Outputs" to start over.

In [ ]:
metadata_api = 'https://api.synopticdata.com/v2/stations/metadata?'

# Convert named network to Synoptic API numeric network IDs (see docs)
synoptic_networks = {
    "NWS+RAWS+HADS":"1,2,106",
    "NWS+RAWS":"1,2",
    "NWS+HADS":"1,106",
    "NWS":"1",
    "RAWS": "2",
    "HADS":"106",
    "ALL":None}

network_query = (f"&network={synoptic_networks[network]}"
                 if synoptic_networks[network] is not None else '')

# Assemble the API query
api_query = (f"{metadata_api}&token={token}&cwa={cwa}" + network_query +
            f"&complete=1&sensorvars=1,obrange=20230118")

# Print the API query to output
print(api_query)

# Get the data from the API
response = requests.get(api_query)
metadata = pd.DataFrame(response.json()['STATION'])

# Remove NaNs and index by network, station ID
metadata = metadata[metadata['MNET_SHORTNAME'].notna()]
metadata = metadata.set_index(['MNET_SHORTNAME', 'STID'])

metadata['LATITUDE'] = metadata['LATITUDE'].astype(float)
metadata['LONGITUDE'] = metadata['LONGITUDE'].astype(float)
metadata['ELEVATION'] = metadata['ELEVATION'].astype(float)

metadata = metadata[metadata['LATITUDE'] >= 31]
metadata = metadata[metadata['LONGITUDE'] <= -103.00]
metadata = metadata[metadata['STATUS'] == 'ACTIVE']

variable_mask = np.array([i for i, md in enumerate(metadata['SENSOR_VARIABLES'])
                                if variable in str(md.keys())])

metadata = metadata.iloc[variable_mask]

print(metadata.shape)
metadata.head()

In [ ]:
network_names = metadata.index.get_level_values(0).unique()

print('Station Counts:')
for nn in network_names:
    print(nn, metadata.loc[nn, :].shape[0])

In [ ]:
print('Station Elevations:')

fig, axs = plt.subplots(network_names.size, 1, sharex=True,
                        figsize=(8, 1.5*network_names.size))

try:
    axs = axs.flatten()
except:
    axs = [axs]

for i, nn in enumerate(network_names):
    axs[i].hist(metadata.loc[nn, :]['ELEVATION'],
             bins=np.arange(0, np.ceil(metadata.ELEVATION.max()/1000)*1000, 500),
                edgecolor='k', alpha=0.5, width=475,
             label=f'{nn} ({metadata.loc[nn, :].shape[0]})')
    axs[i].grid(zorder=-1)
    axs[i].set_xlim(left=0)
    axs[i].legend()

axs[i].set_xlabel('Elevation (ft)')

fig.subplots_adjust(hspace=0.1)
plt.show()

The cell below leverages geopandas and cartopy to plot the stations across a selected CWA (or multiple) with elevation shaded. If the list of networks is longer than the available marker types, they will be plotted together as one.

In [ ]:
geo_metadata = gpd.GeoDataFrame(metadata,
                                geometry=gpd.points_from_xy(
                                    metadata.LONGITUDE, metadata.LATITUDE),
                                crs="EPSG:4326")

cmap = mpl.cm.terrain
vmin = np.floor(metadata.ELEVATION.min()/1000)*1000
vmax = np.ceil(metadata.ELEVATION.max()/1000)*1000
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)

markers = [",","o","v","^","<",">"]

# Create a figure and axes object
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())

for i, nn in enumerate(network_names):
    marker = markers[i] if network_names.size <= len(markers) else ','
    legend = True if ((i == len(network_names)-1)
                        & (network_names.size <= len(markers))) else False

    geo_metadata.loc[nn, :].plot(column='ELEVATION', ax=ax, zorder=10,
                    markersize=75, marker=marker, label=nn,
                    vmin=vmin, vmax=vmax, cmap=cmap, norm=norm,
                    legend=legend,
                    legend_kwds={'label': 'Elevation (ft)', 'shrink':0.5})

# # Add boundaries
ax.add_feature(cfeature.LAND.with_scale('50m'))
ax.add_feature(cfeature.OCEAN.with_scale('50m'))
ax.add_feature(cfeature.LAKES.with_scale('50m'))
ax.add_feature(cfeature.STATES.with_scale('50m'))
ax.coastlines()

plt.title(f'{cwa.upper()} Stations ({network})')

plt.show()